<h1><center>Midterm Election 2018</center></h1>

In [1]:
#Importing packages
import tweepy
import pymongo
import datetime
import pandas as pd

In [2]:
#Variables that contains the user credentials to access Twitter API 
consumer_key = 'XeRPtmT6mBf0Yfd3IjPGhhv5x'
consumer_secret = 'fHzH7CAJ1qujrugS2dZ8ZTkFL102lTL45X8zL5wmfKg5CSXphs'
access_token = '984662830770253824-1W2kGkI2hIKrKJpBMaFEvkbjuV4rdlG'
access_token_secret = 'shgWn5P5bHhIwwKy6qABHEmZ3cvc8odAUHliSA8hBMNMt'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [3]:
#Creating a connection to MongoDB database 'SearchAPITweetsDB'
MONGO_HOST = 'mongodb://localhost/SearchAPITweetsDB'
client = pymongo.MongoClient(MONGO_HOST)
db = client.SearchAPITweetsDB
collection = db.midtermelection

In [4]:
datestr = datetime.date.today().strftime("%Y-%m-%d")

In [5]:
def extract_data(datestr):
    for tweet in tweepy.Cursor(api.search, q = "#MidtermElection2018 -filter:retweets -filter:media -filter:links \
                               -filter:images -filter:native_video -filter:vine -filter:twimg",\
                               lang="en",\
                               since="2018-09-1",\
                               until=datestr).items(500):
        data ={}
        data['created_at'] = tweet.created_at
        data['text'] = tweet.text
        #data['location'] = tweet.user.location

        collection.insert_one(data)

In [6]:
extract_data(datestr)

In [7]:
collection.count()

500

In [8]:
cursor = collection.find({})

In [9]:
DF = pd.DataFrame(list(cursor))

In [10]:
DF.columns

Index(['_id', 'created_at', 'text'], dtype='object')

In [11]:
DF.drop('_id', axis = 1, inplace = True)

In [12]:
DF.to_csv('midtermelection.csv', index = False)